In [6]:
import pandas as pd
import numpy as np
import os
import sys
import glob
from datetime import datetime
# Make `src` dir can be imported
project_root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))  # /aicup-predict-energy-generation
sys.path.append(project_root_path)

from src.utils import choose_device


import matplotlib.pyplot as plt
import seaborn as sns

from src.fe_tools import create_time_features

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [5]:
config_file = './Sean/test_5_L10_pe.json'
config_file_name = config_file.split('/')[2].split('.')[0]
config_file_name

'test_5_L10_pe'

In [2]:
df_raw_data = pd.read_csv('../data/processed_data/combined_data.csv')
df_raw_data['datetime'] = pd.to_datetime(df_raw_data['datetime'])
# make sure the sorting is correct
df_raw_data = df_raw_data.sort_values(by=['device','datetime']).reset_index(drop=True)

df_device = choose_device(df_raw_data, 'L10')

In [40]:
# predicted result
dir = '../pred_results/Sean/'
result_file = [file for file in os.listdir(dir) if file.startswith(config_file_name)][0]
result_file

df_result = pd.read_csv(os.path.join(dir, result_file))
df_result['datetime'] = pd.to_datetime(df_result['datetime'])

In [41]:
df_result.head()

,fold,datetime,y_valid,y_pred,tae
0,0,2024-06-30 07:24:12,12.16,14.809577,2.649577
1,0,2024-06-30 07:25:12,11.84,13.302484,1.462484
2,0,2024-06-30 07:26:12,11.14,11.424655,0.284655
3,0,2024-06-30 07:27:12,10.91,10.292706,0.617294
4,0,2024-06-30 07:28:12,10.54,10.299067,0.240933


In [42]:
def get_target_data(df, error):
    if isinstance(error, (int, float)):
        df_temp = df[df['tae'] > error]

    elif isinstance(error, list) and len(error) == 2:
        df_temp = df[(df['tae'] >= error[0]) & (df['tae'] < error[1])]

    else:
        raise ValueError("Error should be either a single number or a list with two elements.")

    return df_temp

In [47]:
df_error = get_target_data(df_result, error=1000)
df_error

,fold,datetime,y_valid,y_pred,tae
509,0,2024-07-01 11:53:15,1577.31,409.86404,1167.445956
513,0,2024-07-01 11:57:15,1719.90,437.61127,1282.288733
514,0,2024-07-01 11:58:15,1729.46,437.89197,1291.568032
536,0,2024-07-01 12:20:15,1595.78,410.86430,1184.915712
537,0,2024-07-01 12:21:15,1621.41,409.65936,1211.750637
...,...,...,...,...,...
9169,3,2024-07-13 12:41:16,1705.99,557.75256,1148.237437
9170,3,2024-07-13 12:42:16,1722.16,546.22290,1175.937100
9171,3,2024-07-13 12:43:16,1721.86,387.93910,1333.920913
9172,3,2024-07-13 12:44:16,1721.40,384.86703,1336.532965


In [49]:
df = pd.merge(df_error, df_device, how='left', on='datetime')
df

,fold,datetime,y_valid,y_pred,tae,windspeed,pressure,temperature,humidity,sunlight,power,device
0,0,2024-07-01 11:53:15,1577.31,409.86404,1167.445956,0.00,1002.62,49.25,26.24,117758.20,1577.31,L10
1,0,2024-07-01 11:57:15,1719.90,437.61127,1282.288733,0.00,1002.47,49.66,24.44,117758.20,1719.90,L10
2,0,2024-07-01 11:58:15,1729.46,437.89197,1291.568032,0.00,1002.51,49.82,24.14,117758.20,1729.46,L10
3,0,2024-07-01 12:20:15,1595.78,410.86430,1184.915712,0.00,1002.24,50.15,22.22,116951.40,1595.78,L10
4,0,2024-07-01 12:21:15,1621.41,409.65936,1211.750637,0.00,1002.17,49.98,22.31,117393.43,1621.41,L10
...,...,...,...,...,...,...,...,...,...,...,...,...
139,3,2024-07-13 12:41:16,1705.99,557.75256,1148.237437,5.75,1002.33,43.45,34.43,83587.03,1705.99,L10
140,3,2024-07-13 12:42:16,1722.16,546.22290,1175.937100,4.70,1002.45,43.45,32.89,82009.38,1722.16,L10
141,3,2024-07-13 12:43:16,1721.86,387.93910,1333.920913,3.48,1002.47,43.29,34.26,79928.59,1721.86,L10
142,3,2024-07-13 12:44:16,1721.40,384.86703,1336.532965,3.75,1002.42,43.44,34.14,78180.23,1721.40,L10


In [50]:
df.fold.value_counts()

fold
1    65
3    53
0    23
2     2
4     1
Name: count, dtype: int64

In [53]:
df_time_feature = create_time_features(df, input_column='datetime')
df_time_feature

,fold,datetime,y_valid,y_pred,tae,windspeed,pressure,temperature,humidity,sunlight,...,device,date,year,month,day,hour,min,day_of_week,week_of_year,quarter
0,0,2024-07-01 11:53:15,1577.31,409.86404,1167.445956,0.00,1002.62,49.25,26.24,117758.20,...,L10,2024-07-01,2024,7,1,11,53,0,27,3
1,0,2024-07-01 11:57:15,1719.90,437.61127,1282.288733,0.00,1002.47,49.66,24.44,117758.20,...,L10,2024-07-01,2024,7,1,11,57,0,27,3
2,0,2024-07-01 11:58:15,1729.46,437.89197,1291.568032,0.00,1002.51,49.82,24.14,117758.20,...,L10,2024-07-01,2024,7,1,11,58,0,27,3
3,0,2024-07-01 12:20:15,1595.78,410.86430,1184.915712,0.00,1002.24,50.15,22.22,116951.40,...,L10,2024-07-01,2024,7,1,12,20,0,27,3
4,0,2024-07-01 12:21:15,1621.41,409.65936,1211.750637,0.00,1002.17,49.98,22.31,117393.43,...,L10,2024-07-01,2024,7,1,12,21,0,27,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,3,2024-07-13 12:41:16,1705.99,557.75256,1148.237437,5.75,1002.33,43.45,34.43,83587.03,...,L10,2024-07-13,2024,7,13,12,41,5,28,3
140,3,2024-07-13 12:42:16,1722.16,546.22290,1175.937100,4.70,1002.45,43.45,32.89,82009.38,...,L10,2024-07-13,2024,7,13,12,42,5,28,3
141,3,2024-07-13 12:43:16,1721.86,387.93910,1333.920913,3.48,1002.47,43.29,34.26,79928.59,...,L10,2024-07-13,2024,7,13,12,43,5,28,3
142,3,2024-07-13 12:44:16,1721.40,384.86703,1336.532965,3.75,1002.42,43.44,34.14,78180.23,...,L10,2024-07-13,2024,7,13,12,44,5,28,3


In [54]:
df_time_feature.date.value_counts()

date
2024-07-05    54
2024-07-13    53
2024-07-01    11
2024-07-02    10
2024-07-07    10
2024-07-03     2
2024-07-04     1
2024-07-08     1
2024-07-10     1
2024-07-17     1
Name: count, dtype: int64

In [55]:
df_time_feature.hour.value_counts()

hour
12    96
11    40
10     8
Name: count, dtype: int64

In [69]:
df_time_feature.sunlight.value_counts().sort_index(ascending=False).reset_index().head(10)

,sunlight,count
0,117758.20,91
1,117510.23,1
2,117467.10,1
3,117393.43,1
4,117188.59,1
5,117154.45,1
6,116951.40,1
7,115274.91,1
8,113012.65,1
9,112674.84,1


In [88]:
df_time_feature.groupby(['hour','sunlight']).size().reset_index(name='count').sort_values('count', ascending=False).head(10)

,hour,sunlight,count
54,12,117758.20,53
10,11,117758.20,38
1,10,26241.67,1
3,10,26968.33,1
4,10,27495.83,1
5,10,28035.83,1
0,10,26107.50,1
6,10,69434.84,1
7,10,72175.08,1
8,11,54612.50,1


In [91]:
# select the row by definition
df_result_add_time_feature = create_time_features(df_device, input_column='datetime')
df_result_add_time_feature

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device,date,year,month,day,hour,min,day_of_week,week_of_year,quarter
101673,2024-03-01 17:14:06,0.0,1017.48,15.59,94.30,652.92,0.12,L10,2024-03-01,2024,3,1,17,14,4,9,1
101674,2024-03-01 17:14:47,0.0,1017.48,15.66,94.04,682.50,0.12,L10,2024-03-01,2024,3,1,17,14,4,9,1
101675,2024-03-01 17:15:47,0.0,1017.47,15.74,94.10,750.00,0.14,L10,2024-03-01,2024,3,1,17,15,4,9,1
101676,2024-03-01 17:16:47,0.0,1017.46,15.78,94.09,738.33,0.14,L10,2024-03-01,2024,3,1,17,16,4,9,1
101677,2024-03-01 17:17:47,0.0,1017.49,15.80,94.08,660.83,0.12,L10,2024-03-01,2024,3,1,17,17,4,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193008,2024-07-22 05:15:28,0.0,1004.27,27.36,88.72,102.50,0.00,L10,2024-07-22,2024,7,22,5,15,0,30,3
193009,2024-07-22 05:16:28,0.0,1004.25,27.37,88.78,115.00,0.01,L10,2024-07-22,2024,7,22,5,16,0,30,3
193010,2024-07-22 05:17:28,0.0,1004.29,27.37,88.88,129.17,0.01,L10,2024-07-22,2024,7,22,5,17,0,30,3
193011,2024-07-22 05:18:28,0.0,1004.29,27.38,88.88,144.17,0.01,L10,2024-07-22,2024,7,22,5,18,0,30,3


In [92]:
hour_range = [i for i in range(10,13)]
hour_range

[10, 11, 12]

In [122]:
df_sun_hour = df_result_add_time_feature[df_result_add_time_feature['hour']<=12][['datetime','hour','sunlight','date']]

In [123]:
df_sun_hour.sort_values('datetime')

,datetime,hour,sunlight,date
101725,2024-03-02 06:06:47,6,22.50,2024-03-02
101726,2024-03-02 06:07:47,6,28.33,2024-03-02
101727,2024-03-02 06:08:47,6,34.17,2024-03-02
101728,2024-03-02 06:09:47,6,35.83,2024-03-02
101729,2024-03-02 06:10:47,6,37.50,2024-03-02
...,...,...,...,...
193008,2024-07-22 05:15:28,5,102.50,2024-07-22
193009,2024-07-22 05:16:28,5,115.00,2024-07-22
193010,2024-07-22 05:17:28,5,129.17,2024-07-22
193011,2024-07-22 05:18:28,5,144.17,2024-07-22


In [128]:
# The data is inhomogeneous, with record counts varying significantly across dates 
df_sun_hour.date.value_counts().reset_index()

,date,count
0,2024-06-05,488
1,2024-05-30,487
2,2024-06-09,487
3,2024-05-31,486
4,2024-06-08,486
...,...,...
128,2024-03-07,172
129,2024-06-04,96
130,2024-05-01,18
131,2024-07-22,14


In [152]:
# there is a big gap when the count less than 400, so we select the date with more than 400 data
date_for_sunlight_sim = df_sun_hour.date.value_counts().reset_index().query("count > 400").date.astype(str).tolist()

In [153]:
df_sunlight_sim = df_device[df_device['datetime'].dt.date.astype(str).isin(date_for_sunlight_sim)]
df_sunlight_sim

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device
101725,2024-03-02 06:06:47,0.00,1020.13,14.18,94.37,22.50,0.0,L10
101726,2024-03-02 06:07:47,0.00,1020.10,14.19,94.27,28.33,0.0,L10
101727,2024-03-02 06:08:47,0.00,1020.07,14.19,94.35,34.17,0.0,L10
101728,2024-03-02 06:09:47,0.00,1020.07,14.19,94.42,35.83,0.0,L10
101729,2024-03-02 06:10:47,0.00,1020.04,14.20,94.46,37.50,0.0,L10
...,...,...,...,...,...,...,...,...
192994,2024-07-21 18:44:27,0.00,1005.26,32.27,67.49,37.50,0.0,L10
192995,2024-07-21 18:45:27,0.00,1005.24,32.26,67.12,32.50,0.0,L10
192996,2024-07-21 18:46:27,0.52,1005.25,32.24,67.05,28.33,0.0,L10
192997,2024-07-21 18:47:27,0.26,1005.27,32.18,67.28,24.17,0.0,L10


In [154]:
df_sunlight_sim.to_csv('../data/processed_data/sunlight_simulation_data_csv', index=False)